# ETL PROJECT

AIRLINE DELAY CAUSES

- Dataset 1 is about "Airline Information" from folder DOT_dataset contains airline and airport information. Link: https://www.kaggle.com/usdot/flight-delays?select=flights.csv-
- Dataset 2 is about "Airport Information" from folder DOT_dataset contains airline and airport information. Link: https://www.kaggle.com/usdot/flight-delays?select=flights.csv-
- Dataset 3 is about "2015 Airline Delays" from folder Yuanyu_dataset shows airline delay reports between 2009 and 2018. Link: https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018 
- Dataset 4 is about "Airline Delay Reasons" from folder Sharma_dataset gives airline delay reasons. Link: https://www.kaggle.com/anshuls235/airline-delay-causes

## Extract

In [1]:
import numpy as np
import pandas as pd
import time
import datetime
from config import key
from sqlalchemy import create_engine

In [2]:
# Name and Location of the CSV files
file1 = 'Resources/DOT_dataset/datasets_810_1496_airlines.csv'
file2 = 'Resources/DOT_dataset/datasets_810_1496_airports.csv'
file3 = 'Resources/Yuanyu_dataset/2015.csv'
file4 = 'Resources/Sharma_dataset/894233989_32020_056_airline_delay_causes.csv'


In [3]:
# Read CSV files
airline_info = pd.read_csv(file1, encoding="ISO-8859-1")
airport_info = pd.read_csv(file2, encoding="ISO-8859-1")
year_2015 = pd.read_csv(file3, encoding="ISO-8859-1")
delay_causes = pd.read_csv(file4, encoding="ISO-8859-1")

In [4]:
# Show heads of files downloaded
print(airline_info.head())
print(airport_info.head())
print(year_2015.head())
print(delay_causes.head())

  IATA_CODE                 AIRLINE
0        UA   United Air Lines Inc.
1        AA  American Airlines Inc.
2        US         US Airways Inc.
3        F9  Frontier Airlines Inc.
4        B6         JetBlue Airways
  IATA_CODE                              AIRPORT         CITY STATE COUNTRY  \
0       ABE  Lehigh Valley International Airport    Allentown    PA     USA   
1       ABI             Abilene Regional Airport      Abilene    TX     USA   
2       ABQ    Albuquerque International Sunport  Albuquerque    NM     USA   
3       ABR            Aberdeen Regional Airport     Aberdeen    SD     USA   
4       ABY   Southwest Georgia Regional Airport       Albany    GA     USA   

   LATITUDE  LONGITUDE  
0  40.65236  -75.44040  
1  32.41132  -99.68190  
2  35.04022 -106.60919  
3  45.44906  -98.42183  
4  31.53552  -84.19447  
      FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  CRS_DEP_TIME  \
0  2015-01-01         NK                195    MCO  FLL          2147   
1  2015-01-01

## Transforming Data

### 1. Airline Codes

In [5]:
airline_info

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [6]:
# Primary Key Column Values
airline_info["IATA_CODE"].unique()

array(['UA', 'AA', 'US', 'F9', 'B6', 'OO', 'AS', 'NK', 'WN', 'DL', 'EV',
       'HA', 'MQ', 'VX'], dtype=object)

### 2. Airport Information

In [7]:
airport_info.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [8]:
airport_info.count()

IATA_CODE    322
AIRPORT      322
CITY         322
STATE        322
COUNTRY      322
LATITUDE     319
LONGITUDE    319
dtype: int64

In [9]:
airport_info.dropna()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


### 3. Airline Delay Table (2015)

In [10]:
year_2015.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2015-01-01,NK,195,MCO,FLL,2147,2143.0,-4.0,15.0,2158.0,...,63.0,62.0,40.0,177.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,NK,197,LGA,FLL,1050,1104.0,14.0,20.0,1124.0,...,194.0,179.0,150.0,1076.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,NK,198,FLL,MCO,700,712.0,12.0,19.0,731.0,...,57.0,61.0,32.0,177.0,0.0,0.0,16.0,0.0,0.0,NaN
3,2015-01-01,NK,199,IAH,LAS,2240,2251.0,11.0,8.0,2259.0,...,196.0,176.0,164.0,1222.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,NK,200,IAH,ORD,623,620.0,-3.0,15.0,635.0,...,152.0,140.0,115.0,925.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Primary Key Column Values
year_2015["OP_CARRIER"].unique()

array(['NK', 'MQ', 'OO', 'EV', 'HA', 'UA', 'US', 'VX', 'WN', 'B6', 'F9',
       'AA', 'AS', 'DL'], dtype=object)

In [12]:
year_2015.dtypes

FL_DATE                 object
OP_CARRIER              object
OP_CARRIER_FL_NUM        int64
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY              float64
TAXI_OUT               float64
WHEELS_OFF             float64
WHEELS_ON              float64
TAXI_IN                float64
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
CANCELLATION_CODE       object
DIVERTED               float64
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed: 27            float64
dtype: object

In [13]:
year = year_2015[["FL_DATE","OP_CARRIER","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","CANCELLED","DIVERTED","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]].copy()

In [14]:
year.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2015-01-01,NK,MCO,FLL,2143.0,-4.0,2245.0,-5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,NK,LGA,FLL,1104.0,14.0,1403.0,-1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,NK,FLL,MCO,712.0,12.0,813.0,16.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0
3,2015-01-01,NK,IAH,LAS,2251.0,11.0,2347.0,-9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,NK,IAH,ORD,620.0,-3.0,840.0,-15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [15]:
year["FL_DATE"] = pd.to_datetime(year['FL_DATE'])

In [16]:
# year['DEP_TIME'] = year['DEP_TIME'].astype(str)
# year["DEP_TIME"] = pd.to_datetime(year["DEP_TIME"], format="%H:%M" )

In [17]:

#year["FL_DATE"] = pd.to_datetime(year['FL_DATE'])
#print("{0:.2f}".format(year['DEP_TIME']))
#print(type(float("{0:3.1f}".format(year['DEP_TIME']))))
#year["DEP_TIME"] = pd.to_datetime( ), format="%H:%M")
#year["ARR_TIME"] = datetime.datetime.strftime(datetime.datetime.utcfromtimestamp(year['ARR_TIME']), "%H:%M")

In [18]:
#year["DEP_TIME"] = pd.to_datetime(year['DEP_TIME'], format="%H:%M")

In [19]:
year = year.dropna()

In [20]:
year.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2,2015-01-01,NK,FLL,MCO,712.0,12.0,813.0,16.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0
15,2015-01-01,NK,FLL,ACY,815.0,15.0,1043.0,18.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
18,2015-01-01,NK,MSP,FLL,127.0,12.0,607.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
20,2015-01-01,NK,FLL,LGA,709.0,-3.0,1019.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0
23,2015-01-01,MQ,BNA,ORD,1927.0,17.0,2156.0,66.0,0.0,0.0,1.0,0.0,49.0,0.0,16.0


In [21]:
year.dtypes

FL_DATE                datetime64[ns]
OP_CARRIER                     object
ORIGIN                         object
DEST                           object
DEP_TIME                      float64
DEP_DELAY                     float64
ARR_TIME                      float64
ARR_DELAY                     float64
CANCELLED                     float64
DIVERTED                      float64
CARRIER_DELAY                 float64
WEATHER_DELAY                 float64
NAS_DELAY                     float64
SECURITY_DELAY                float64
LATE_AIRCRAFT_DELAY           float64
dtype: object

### 4. Delay Causes Table

In [22]:
delay_causes.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2004,1,DL,Delta Air Lines Inc.,PBI,"West Palm Beach/Palm Beach, FL: Palm Beach Int...",650.0,126.0,21.06,6.44,...,45.92,4.0,0.0,5425.0,881.0,397.0,2016.0,15.0,2116.0,NaN
1,2004,1,DL,Delta Air Lines Inc.,PDX,"Portland, OR: Portland International",314.0,61.0,14.09,2.61,...,10.05,30.0,3.0,2801.0,478.0,239.0,1365.0,0.0,719.0,NaN
2,2004,1,DL,Delta Air Lines Inc.,PHL,"Philadelphia, PA: Philadelphia International",513.0,97.0,27.60,0.42,...,17.12,15.0,0.0,4261.0,1150.0,16.0,2286.0,0.0,809.0,NaN
3,2004,1,DL,Delta Air Lines Inc.,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",334.0,78.0,20.14,2.02,...,16.45,3.0,1.0,3400.0,1159.0,166.0,1295.0,0.0,780.0,NaN
4,2004,1,DL,Delta Air Lines Inc.,PIT,"Pittsburgh, PA: Pittsburgh International",217.0,47.0,8.08,0.44,...,16.59,4.0,1.0,1737.0,350.0,28.0,522.0,0.0,837.0,NaN


In [23]:
delay_causes["Unnamed: 21"].unique()

array([nan])

In [24]:
delay_causes.dtypes

year                     int64
 month                   int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
 weather_ct            float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
 arr_delay             float64
 carrier_delay         float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
Unnamed: 21            float64
dtype: object

In [25]:
delay_causes_2015 = delay_causes.loc[delay_causes["year"] == 2015]

In [26]:
delay_causes_2015 = delay_causes_2015[[" month","carrier","carrier_name","arr_cancelled","arr_diverted"," arr_delay"," carrier_delay","weather_delay","nas_delay","security_delay","late_aircraft_delay"]]
delay_causes_2015 = delay_causes_2015.rename(columns={" month":"month"," arr_delay":"arr_delay"," carrier_delay":"carrier_delay"})

In [27]:
delay_causes_2015.head()

,month,carrier,carrier_name,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
122500,1,AA,American Airlines Inc.,86.0,3.0,20055.0,5273.0,999.0,6358.0,0.0,7425.0
122501,1,AA,American Airlines Inc.,41.0,4.0,25261.0,10914.0,1460.0,3293.0,42.0,9552.0
122502,1,AA,American Airlines Inc.,203.0,6.0,167313.0,66714.0,5055.0,24137.0,123.0,71284.0
122503,1,AA,American Airlines Inc.,3.0,0.0,1776.0,1207.0,0.0,188.0,0.0,381.0
122504,1,AA,American Airlines Inc.,0.0,4.0,4175.0,2602.0,0.0,523.0,0.0,1050.0


In [28]:
# Just Curiosity: How many delays have occured by each airline?

# Number of cancelled flights by airport in 2015
delay_causes_2015.groupby('carrier_name')['arr_cancelled'].sum()

carrier_name
Alaska Airlines Inc.          669.0
American Airlines Inc.      10919.0
Delta Air Lines Inc.         3824.0
Envoy Air                   15025.0
ExpressJet Airlines Inc.    15231.0
Frontier Airlines Inc.        588.0
Hawaiian Airlines Inc.        171.0
JetBlue Airways              4276.0
SkyWest Airlines Inc.        9960.0
Southwest Airlines Co.      16043.0
Spirit Air Lines             2004.0
US Airways Inc.              4067.0
United Air Lines Inc.        6573.0
Virgin America                534.0
Name: arr_cancelled, dtype: float64

In [29]:
# Number of flights diverted by airports in 2015
delay_causes_2015.groupby('carrier_name')['arr_delay'].sum()

carrier_name
Alaska Airlines Inc.         1074977.0
American Airlines Inc.       7831435.0
Delta Air Lines Inc.         7229891.0
Envoy Air                    3812234.0
ExpressJet Airlines Inc.     6850156.0
Frontier Airlines Inc.       1583839.0
Hawaiian Airlines Inc.        341192.0
JetBlue Airways              3610200.0
SkyWest Airlines Inc.        6506580.0
Southwest Airlines Co.      12371384.0
Spirit Air Lines             2162991.0
US Airways Inc.              1838065.0
United Air Lines Inc.        6775806.0
Virgin America                663117.0
Name: arr_delay, dtype: float64

In [30]:
delay_causes_2015.groupby(['carrier_name'])[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].sum()

,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
carrier_name,,,,,
Alaska Airlines Inc.,347425.0,38832.0,301478.0,5825.0,381417.0
American Airlines Inc.,2753994.0,467420.0,1760561.0,16158.0,2833302.0
Delta Air Lines Inc.,2707569.0,602901.0,1779383.0,3910.0,2136128.0
Envoy Air,1055033.0,402305.0,930774.0,7049.0,1417073.0
ExpressJet Airlines Inc.,2363973.0,169313.0,1687894.0,0.0,2628976.0
Frontier Airlines Inc.,346950.0,21616.0,581234.0,0.0,634039.0
Hawaiian Airlines Inc.,196422.0,11429.0,6241.0,401.0,126699.0
JetBlue Airways,1074056.0,115770.0,991461.0,11417.0,1417496.0
SkyWest Airlines Inc.,2043703.0,250325.0,1333972.0,9896.0,2868684.0


### Loading DataFrames into Database

In [31]:
airline_info.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [32]:
airport_info.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [33]:
year.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2,2015-01-01,NK,FLL,MCO,712.0,12.0,813.0,16.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0
15,2015-01-01,NK,FLL,ACY,815.0,15.0,1043.0,18.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
18,2015-01-01,NK,MSP,FLL,127.0,12.0,607.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
20,2015-01-01,NK,FLL,LGA,709.0,-3.0,1019.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0
23,2015-01-01,MQ,BNA,ORD,1927.0,17.0,2156.0,66.0,0.0,0.0,1.0,0.0,49.0,0.0,16.0


In [34]:
delay_causes_2015.head()

,month,carrier,carrier_name,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
122500,1,AA,American Airlines Inc.,86.0,3.0,20055.0,5273.0,999.0,6358.0,0.0,7425.0
122501,1,AA,American Airlines Inc.,41.0,4.0,25261.0,10914.0,1460.0,3293.0,42.0,9552.0
122502,1,AA,American Airlines Inc.,203.0,6.0,167313.0,66714.0,5055.0,24137.0,123.0,71284.0
122503,1,AA,American Airlines Inc.,3.0,0.0,1776.0,1207.0,0.0,188.0,0.0,381.0
122504,1,AA,American Airlines Inc.,0.0,4.0,4175.0,2602.0,0.0,523.0,0.0,1050.0


In [35]:
connection_string = "postgres:"+key+"@localhost:5432/airline_db"
engine = create_engine(f'postgresql://{connection_string}')

In [36]:
engine.table_names()

['airline_info', 'airport_info', 'year', 'delay_causes_2015']

In [37]:
airline_info.to_sql(name='airline_info', con=engine, if_exists='append', index=False)
airport_info.to_sql(name='airport_info', con=engine, if_exists='append', index=False)

In [38]:
pd.read_sql_query('select * from airline_info', con=engine).head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [61]:
pd.read_sql_query('select * from airport_info', con=engine).head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.652359999999994,-75.4404
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60918999999998
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.449059999999996,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [40]:
year.to_sql(name='year', con=engine, if_exists='append', index=False)

In [41]:
pd.read_sql_query('select * from year limit 5', con=engine).head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2015-01-01,NK,FLL,MCO,712.0,12.0,813.0,16.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0
1,2015-01-01,NK,FLL,ACY,815.0,15.0,1043.0,18.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
2,2015-01-01,NK,MSP,FLL,127.0,12.0,607.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
3,2015-01-01,NK,FLL,LGA,709.0,-3.0,1019.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0
4,2015-01-01,MQ,BNA,ORD,1927.0,17.0,2156.0,66.0,0.0,0.0,1.0,0.0,49.0,0.0,16.0


In [ ]:
year.to_sql(name='delay_causes_2015', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from delay_causes_2015 limit 5', con=engine).head()

In [64]:
pd.read_sql_query('select y."OP_CARRIER",a."AIRLINE", y."ORIGIN", y."DEST", y."DEP_DELAY", y."ARR_DELAY" from airline_info as a join year as y on a."IATA_CODE"=y."OP_CARRIER"', con=engine)

,OP_CARRIER,AIRLINE,ORIGIN,DEST,DEP_DELAY,ARR_DELAY
0,NK,Spirit Air Lines,FLL,MCO,12.0,16.0
1,NK,Spirit Air Lines,FLL,MCO,12.0,16.0
2,NK,Spirit Air Lines,FLL,MCO,12.0,16.0
3,NK,Spirit Air Lines,FLL,MCO,12.0,16.0
4,NK,Spirit Air Lines,FLL,MCO,12.0,16.0
...,...,...,...,...,...,...
21268775,MQ,American Eagle Airlines Inc.,LCH,DFW,87.0,82.0
21268776,MQ,American Eagle Airlines Inc.,LCH,DFW,87.0,82.0
21268777,MQ,American Eagle Airlines Inc.,LCH,DFW,87.0,82.0
21268778,MQ,American Eagle Airlines Inc.,LCH,DFW,87.0,82.0
